<a href="https://colab.research.google.com/github/giovanadelucca/aquisicao-pre-processamento-e-exploracao-de-dados/blob/main/trabalho_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Configurações iniciais e leitura do conjunto de dados**

**Importação de bibliotecas**

In [23]:
import pandas as pd

**Leitura do conjunto de dados de gasolina e etanol**

[Link da fonte dos dados](https://dados.gov.br/dataset/serie-historica-de-precos-de-combustiveis-por-revenda)

In [24]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [25]:
drive_path = '/content/drive/MyDrive/Datasets/'

df = pd.read_csv(drive_path + 'dataset-completo-2018-2020.csv')
df.head(2)

,Região - Sigla,Estado - Sigla,Município,Revenda,Produto,Data da Coleta,Valor de Venda,Valor de Compra,Unidade de Medida,Bandeira,Instalação - Código,CNPJ da Revenda
0,CO,DF,BRASILIA,BRASAL COMBUSTIVEIS LTDA,ETANOL,2018-07-03,3.399,NaN,R$ / litro,RAIZEN,NaN,NaN
1,CO,DF,BRASILIA,GEOPETROS GEOVANI PETROLEOS E DERIVADOS LTDA,ETANOL,2018-07-03,3.489,3.017,R$ / litro,PETROBRAS DISTRIBUIDORA S.A.,NaN,NaN


### **Aplicação de tratamento de dados simples**

**Renomeação de colunas**

In [26]:
df.columns = ['regiao_sigla', 'estado_sigla', 'municipio', 'revenda', 'produto', 
              'data_coleta', 'valor_venda', 'valor_compra', 'unidade_medida', 'bandeira', 'instalacao_codigo', 'cnpj_revenda']

df.head(2)

,regiao_sigla,estado_sigla,municipio,revenda,produto,data_coleta,valor_venda,valor_compra,unidade_medida,bandeira,instalacao_codigo,cnpj_revenda
0,CO,DF,BRASILIA,BRASAL COMBUSTIVEIS LTDA,ETANOL,2018-07-03,3.399,NaN,R$ / litro,RAIZEN,NaN,NaN
1,CO,DF,BRASILIA,GEOPETROS GEOVANI PETROLEOS E DERIVADOS LTDA,ETANOL,2018-07-03,3.489,3.017,R$ / litro,PETROBRAS DISTRIBUIDORA S.A.,NaN,NaN


**Verificação de dados faltantes**

In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1174501 entries, 0 to 1174500
Data columns (total 12 columns):
 #   Column             Non-Null Count    Dtype  
---  ------             --------------    -----  
 0   regiao_sigla       1174501 non-null  object 
 1   estado_sigla       1174501 non-null  object 
 2   municipio          1174501 non-null  object 
 3   revenda            1174501 non-null  object 
 4   produto            1174501 non-null  object 
 5   data_coleta        1174501 non-null  object 
 6   valor_venda        1174501 non-null  float64
 7   valor_compra       428690 non-null   float64
 8   unidade_medida     1174501 non-null  object 
 9   bandeira           1174501 non-null  object 
 10  instalacao_codigo  230329 non-null   float64
 11  cnpj_revenda       695746 non-null   float64
dtypes: float64(4), object(8)
memory usage: 107.5+ MB


É possível verificar que as colunas **valor_compra**, **intalacao_codigo** e **cnpj_revenda** possuem dados faltantes. Entretanto, nenhuma dessas colunas acrescenta relevância ao tipo de análise que se pretende implementar. Portanto, essas colunas podem ser removidas.

**Remoção de colunas**

In [28]:
df.drop(['valor_compra', 'instalacao_codigo', 'cnpj_revenda'], axis=1, inplace=True)
df.head(2)

,regiao_sigla,estado_sigla,municipio,revenda,produto,data_coleta,valor_venda,unidade_medida,bandeira
0,CO,DF,BRASILIA,BRASAL COMBUSTIVEIS LTDA,ETANOL,2018-07-03,3.399,R$ / litro,RAIZEN
1,CO,DF,BRASILIA,GEOPETROS GEOVANI PETROLEOS E DERIVADOS LTDA,ETANOL,2018-07-03,3.489,R$ / litro,PETROBRAS DISTRIBUIDORA S.A.


**Mapeamento e renomeação da coluna regiao_sigla**

In [29]:
df['regiao_sigla'].value_counts()

SE    589644
NE    231758
S     181805
CO    111283
N      60011
Name: regiao_sigla, dtype: int64

Com a contagem de dados realizada na célula anterior, é possível observar que n
a coluna **regiao_sigla** de fato só contém dados referentes às 5 regiões do Brasil. Assim sendo, para facilitar a interpretação e visualização dos dados, pode-se fazer um mapeamento por meio de dicionário e transformar as siglas nos próprios nomes das regiões.

In [30]:
df['regiao_sigla'] = df['regiao_sigla'].map({
    'N': 'NORTE',
    'NE': 'NORDESTE',
    'CO': 'CENTRO_OESTE',
    'SE': 'SUDENTES',
    'S': 'SUL'
})

df.rename(columns={'regiao_sigla':'regiao'}, inplace=True)

df.head(2)

,regiao,estado_sigla,municipio,revenda,produto,data_coleta,valor_venda,unidade_medida,bandeira
0,CENTRO_OESTE,DF,BRASILIA,BRASAL COMBUSTIVEIS LTDA,ETANOL,2018-07-03,3.399,R$ / litro,RAIZEN
1,CENTRO_OESTE,DF,BRASILIA,GEOPETROS GEOVANI PETROLEOS E DERIVADOS LTDA,ETANOL,2018-07-03,3.489,R$ / litro,PETROBRAS DISTRIBUIDORA S.A.


**Tratamento da coluna unidade_medida**

In [34]:
df['unidade_medida'].value_counts()

REAIS_LITRO    1174501
Name: unidade_medida, dtype: int64

Com a contagem de dados realizada na célula anterior, é possível observar que a coluna **unidade_medida** de fato só contém dados que informam que todos os valores de venda de gasolina e etanol do conjunto de dados é sempre da unidade reais por litro. Para facilitar a interpretação e visualização dos dados, pode-se substituir a representação desta informação vinda da fonte dos dados por uma mais simples, sem caracteres especiais, sem espaços e com letras maiúsculas.

In [31]:
df['unidade_medida'] = df['unidade_medida'].map({'R$ / litro': 'REAIS_LITRO'})

df.head(2)

,regiao,estado_sigla,municipio,revenda,produto,data_coleta,valor_venda,unidade_medida,bandeira
0,CENTRO_OESTE,DF,BRASILIA,BRASAL COMBUSTIVEIS LTDA,ETANOL,2018-07-03,3.399,REAIS_LITRO,RAIZEN
1,CENTRO_OESTE,DF,BRASILIA,GEOPETROS GEOVANI PETROLEOS E DERIVADOS LTDA,ETANOL,2018-07-03,3.489,REAIS_LITRO,PETROBRAS DISTRIBUIDORA S.A.


**Alteração de tipo da coluna data_coleta para datetime**

In [32]:
df['data_coleta'] = pd.to_datetime(df['data_coleta'])
df.head(2)

,regiao,estado_sigla,municipio,revenda,produto,data_coleta,valor_venda,unidade_medida,bandeira
0,CENTRO_OESTE,DF,BRASILIA,BRASAL COMBUSTIVEIS LTDA,ETANOL,2018-07-03,3.399,REAIS_LITRO,RAIZEN
1,CENTRO_OESTE,DF,BRASILIA,GEOPETROS GEOVANI PETROLEOS E DERIVADOS LTDA,ETANOL,2018-07-03,3.489,REAIS_LITRO,PETROBRAS DISTRIBUIDORA S.A.


**Separação do conjunto de dados em relação ao produto**

In [35]:
df['produto'].value_counts()

GASOLINA    622327
ETANOL      552174
Name: produto, dtype: int64

In [44]:
df_etanol = df[df['produto'] == 'ETANOL']
df_gasolina = df[df['produto'] == 'GASOLINA']

In [38]:
df_etanol.head(2)

,regiao,estado_sigla,municipio,revenda,produto,data_coleta,valor_venda,unidade_medida,bandeira
0,CENTRO_OESTE,DF,BRASILIA,BRASAL COMBUSTIVEIS LTDA,ETANOL,2018-07-03,3.399,REAIS_LITRO,RAIZEN
1,CENTRO_OESTE,DF,BRASILIA,GEOPETROS GEOVANI PETROLEOS E DERIVADOS LTDA,ETANOL,2018-07-03,3.489,REAIS_LITRO,PETROBRAS DISTRIBUIDORA S.A.


In [46]:
df_gasolina.head(2)

,regiao,estado_sigla,municipio,revenda,produto,data_coleta,valor_venda,unidade_medida,bandeira
22282,CENTRO_OESTE,DF,BRASILIA,BRASAL COMBUSTIVEIS LTDA,GASOLINA,2018-07-03,4.399,REAIS_LITRO,RAIZEN
22283,CENTRO_OESTE,DF,BRASILIA,GEOPETROS GEOVANI PETROLEOS E DERIVADOS LTDA,GASOLINA,2018-07-03,4.499,REAIS_LITRO,PETROBRAS DISTRIBUIDORA S.A.


In [48]:
if len(df_etanol) + len(df_gasolina) == len(df): print('A soma das quantidades de exemplos dos conjuntos de dados de etanol e gasolina é igual à quantidade total de exemplos.')

A soma das quantidades de exemplos dos conjuntos de dados de etanol e gasolina é igual à quantidade total de exemplos.
